In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

FFNN den farklı olan şeyler şunlardır:

Adam: Öğrenme oranını otomatik ayarlayabilen popüler bir optimizasyon algoritması.

EarlyStopping, ReduceLROnPlateau: Eğitim sürecini daha verimli hale getirmek için kullanılan callback’ler. Overfitting engellemek, fazla epoch çalıştırmamak amacıyla durdurma ya da öğrenme hızını (learning rate) düşürme yapar.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

timesteps = 2000
time = np.linspace(0, 1, timesteps)

SoC = 100 * (1 - np.exp(-5 * time)) + np.random.normal(0, 2, timesteps)
SoC = np.clip(SoC, 0, 100)

SoH = 100 * np.exp(-0.002 * time * timesteps) + np.random.normal(0, 0.5, timesteps)
SoH = np.clip(SoH, 0, 100)

FFNN den farklı olarak np.clip(SoC, 0, 100) kullandık. Bu kod değerlerin 0 ile 100 arasında kalmasını sağlıyor.

In [ ]:
voltage = np.random.uniform(10, 14, timesteps)
current = np.random.uniform(-50, 50, timesteps)
temperature = np.random.uniform(15, 45, timesteps)

10-14 arasında Volt değerleri, -50-50 arasında akım değerleri ve 15-45 derece arasında sıcaklık değerleri oluşturur.

LSTM örneğinde daha gerçekçi bir senaryo için bataryayı etkileyebilecek voltaj, akım ve sıcaklık gibi ek parametreler de modele dahil edilmiş. Böylece LSTM zaman serisi verisindeki SoC, SoH dalgalanmalarının yanı sıra bu sensör verilerini de kullanarak daha iyi bir tahmin yapabilir.

In [ ]:
data = np.column_stack((SoC, SoH, voltage, current, temperature))

Neden tek bir 2D dizi oluşturuyoruz?

Modelin girdi olarak kullanacağı bütün değişkenleri (SoC, SoH, voltage, current, temperature) aynı tabloda (matriste) birleştiriyoruz.

Bu şekilde, her satır bir zaman adımındaki tüm sensör bilgilerini (ve hedef değişkenleri) içeriyor.

Daha sonra bu tabloyu LSTM/NN gibi modellere beslerken tek bir veri yapısı üzerinden işlem yapmak kolaylaşır.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

Burdaki kodun FFNN den bir farkı yok. Sadece veriye ekstra sensör verileri eklmeiş olduk

In [ ]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])     
        y.append(data[i + seq_length, :2])   
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled, seq_length)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

LSTM zaman serisi verisiyle (ardışık verilerle) çalışır.

seq_length = 30 demek, model her seferinde 30 zaman adımını input olarak alır, 31. zamanın SoC ve SoH’sini tahmin etmeye çalışır.

X boyutu (örnek_sayısı, 30, 5) ⇒ 30 zaman adımı, her adımda 5 özelliğimiz var (SoC, SoH, voltage, current, temperature).

y boyutu (örnek_sayısı, 2) ⇒ Sadece SoC ve SoH değerlerini tahmin ediyoruz.

Ayno şekilde burada da verinin %80 i eğitim %20 si tes için kullanılıyor

In [ ]:
model = Sequential([
    Input(shape=(seq_length, X.shape[2])),

    LSTM(64, return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),

    LSTM(32, return_sequences=False),
    BatchNormalization(),
    Dropout(0.2),

    Dense(16, activation='relu'),
    Dense(2, activation='sigmoid')
])

Input(shape=(seq_length, X.shape[2]))

X.shape bir tuple (demet) döndürür, örneğin (num_samples, seq_length, num_features).

X.shape[0] → örnek sayısı (num_samples),

X.shape[1] → zamansal uzunluk (seq_length),

X.shape[2] → her zaman adımında kaç özelliğiniz (feature) olduğu (num_features).

Burada seq_length=30 zaman adımı (örneğin son 30 ölçüm) ve X.shape[2]=5 özellik (SoC, SoH, voltage, current, temperature) var.

Yani model, her bir örnek için 30×5 boyutunda bir zaman serisi alacak.
----------------------------------------------------------------------------
LSTM(64, return_sequences=True):

İlk LSTM katmanı, 64 nöron.

return_sequences=True diyerek, bir sonraki LSTM katmanına tüm zaman adımlarının çıktılarını aktarıyoruz.
------------------------------------------------------------------
LSTM(32, return_sequences=False)

32: Bu LSTM katmanında 32 adet nöron (LSTM hücresi) vardır. Daha yüksek değer, daha fazla parametre ve daha yüksek öğrenme kapasitesi anlamına gelir; ancak eğitim süresini de uzatabilir.
---------------------
return_sequences=False:

LSTM katmanları, varsayılan olarak her zaman adımındaki çıktıyı döndürebilir (True) veya sadece son zaman adımının çıktısını döndürebilir (False).

Burada False olduğu için, katmandan sadece son zaman adımında elde edilen çıktı döner. Bu genellikle zaman serisi tahmini (regresyon) veya sınıflandırma gibi problemlerde, “en son adımın özet bilgisini” almak istediğimizde tercih edilir.

Bir önceki LSTM katmanında return_sequences=True kullanılmışsa, o katmandan her bir zaman adımına ait çıktı alınır. Bu ikinci LSTM katmanı ise, o çıktıları (zaman serisi) teker teker okuyarak sonunda tek bir vektör üretir (son gizli durum – final hidden state). Bu vektör, zaman serisinin bütün bilgisini içinde barındırdığı varsayılır.
----------------------------------------

BatchNormalization()

LSTM katmanından çıkan veriyi, her mini-batch’te ortalama ve standart sapma açısından yeniden ölçekler (normalleştirir).

Amaç, eğitimi hızlandırmak ve dağılımı daha kararlı hale getirmektir. Özellikle LSTM gibi derin yapılarda, katmanlar arası dağılımların sürekli değişmesi “iç dağılım kayması (internal covariate shift)” yaratabilir; batch normalization bu etkiyi hafifletir.

Daha istikrarlı gradyanlar sayesinde model daha hızlı yakınsar, öğrenme oranına daha az hassas hale gelir.
----------------------------------------------
Tam bağlı katman nedir ?

Tam bağlı katman (Fully Connected Layer veya Dense Layer), önceki katmandaki her nöronun, bu katmandaki her nörona ayrı bir ağırlık (weight) ve bias ile bağlanması anlamına gelir. Yani girişteki her birim, katman içindeki tüm nöronlara etki eder. Bu sayede katmanda çok sayıda parametre bulunur ve ağ, giriş sinyalindeki her özelliği öğrenip birleştirebilecek esnekliğe sahip olur. 

-----
Dense(16, activation='relu')

LSTM katman(lar)ından gelen son çıktıyı (yani zaman serisinin işlenmiş hâlini) 16 nöron içeren “tam bağlı (fully connected)” bir katmana veriyoruz.

Burada ReLU (Rectified Linear Unit) aktivasyon fonksiyonu kullanmak, doğrusal olmayan bir dönüşüm sağlayarak modelin öğrenebileceği temsil gücünü artırır.

Amaç, LSTM’den gelen bilgiyi “ek bir katmanla” işleyerek, SoC ve SoH tahmini için daha rafine bir özellik (feature) çıkarımı yapmaktır.

---------------------------------------------------------
Dense(2, activation='sigmoid')

Modelin çıkış katmanı: 2 nöron içerir, çünkü iki adet değer (SoC ve SoH) tahmin etmek istiyoruz.

activation='sigmoid': Her nöron, çıktıyı 0 ile 1 arasına sıkıştırır.

LSTM kodumuzda veriyi 0-1 aralığına ölçeklemiştik (MinMaxScaler). Dolayısıyla “modelin çıktısı” da 0-1 arasında olsun istiyoruz.

Tahmin sonrasında “inverse_transform” uygulayarak bu değerleri tekrar 0–100 yüzdelik aralığına dönüştürüp SoC ve SoH olarak yorumlayabiliyoruz.
---------------------------------------------------------
Kısacası, son iki Dense katmanı:

Önce 16 nöronla LSTM çıktılarını daha ayrıntılı işliyor (ReLU aktivasyonla),

Ardından 2 nöronla (sigmoid) SoC ve SoH’yi 0-1 aralığında nihai olarak tahmin ediyor.
-----------------------------------------------------------


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=tf.keras.losses.Huber())


FFNN den farklı olarak burada Huber kullandık. Huber Loss: MSE ve MAE arasında bir denge sağlar, uç değerlerden (outlier) çok fazla etkilenmez, stabil bir öğrenme sunar.

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

history = model.fit(X_train, y_train,
                    epochs=30,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks,
                    verbose=1)

ReduceLROnPlateau:

monitor='val_loss' ⇒ validasyon kaybı (val_loss) gelişmiyorsa learning rate’i factor=0.5 oranında yarıya indirir.

patience=3 ⇒ 3 epoch boyunca iyileşme olmazsa bu işlemi yapar.
--------------------------------
EarlyStopping:

monitor='val_loss', patience=6 ⇒ 6 epoch boyunca val_loss iyileşmiyorsa eğitim durur.

restore_best_weights=True ⇒ eğitim boyunca en iyi doğrulama skoruna (val_loss en düşük) sahip olan ağırlıkları al.
---------------------------------
epochs=30: Model veriyi 30 kez tarayacak veya early stopping devreye girerse daha önce duracak.
---------------------------------------
batch_size=32: Her adımda 32 örnekle güncelleme yapar (mini-batch).
---------------------------------------
validation_data=(X_test, y_test): Eğitim sırasında test setiyle de doğrulama kaybını hesaplıyor.

In [ ]:
predictions = model.predict(X_test)

predictions_rescaled = scaler.inverse_transform(
    np.column_stack((predictions, np.zeros((len(predictions), 3))))
)[:, :2]

predictions_rescaled[:, 0] = np.clip(predictions_rescaled[:, 0], 0, 100)
predictions_rescaled[:, 1] = np.clip(predictions_rescaled[:, 1], 0, 100)


y_test_rescaled = scaler.inverse_transform(
    np.column_stack((y_test, np.zeros((len(y_test), 3))))
)[:, :2]

model.predict(X_test): Test verisi için modelin çıkardığı [SoC, SoH] tahminleri (0–1 aralığında).
---------------------------------
inverse_transform: Daha önce MinMaxScaler ile ölçeklenen veriyi, orijinal skala (0–100 aralığı) düzeyine geri çevirir.
--------------------------------------
Neden np.column_stack((predictions, np.zeros((len(predictions), 3)))) yapıyoruz?

Orijinal veri 5 sütundan oluşuyordu (SoC, SoH, voltage, current, temperature). Şimdi sadece 2 sütunumuz (SoC, SoH tahmini) var. inverse_transform fonksiyonu, aynı sayıdaki sütuna ihtiyaç duyar. Bu yüzden kalan 3 sütunu “0” olarak ekleyip, inverse_transform sonrasında ilk 2 sütunu alıyoruz.
------------------------------------------
Tahminlerin 0–100 aralığında kalmasını sağlıyoruz (SoC, SoH fiziksel olarak bu aralığı aşmamalı).
-------------------------------------------------
Gerçek SoC, SoH değerlerini de aynı şekilde orijinal ölçeğe dönüştürüyoruz ki tahminle karşılaştıralım.

3 ve 2 değerlerini neye göre yazdık ?

3, geriye kalan 3 sütunun (voltage, current, temperature) yerini sıfırla doldurduğumuzu belirtir.

2, inverse_transform sonrası elde edilen (n, 5) matristen sadece ilk 2 sütunu (SoC, SoH) almaya yarar.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:, 0], label="Actual SoC", linestyle="dashed")
plt.plot(predictions_rescaled[:, 0], label="Predicted SoC", linestyle="solid")
...
